# Importing Libraries and Initializations

In [ ]:
# Install huggingface specific libraries
! pip install transformers datasets evaluate --quiet

In [ ]:
# Common libraries' imports
import json
import time
import numpy as np
import pandas as pd
import warnings
from time import gmtime, strftime
from random import randrange
warnings.filterwarnings('ignore')

# Check versions
from platform import python_version
import torch
import sagemaker
import transformers
import datasets
print('Pytorch version: ', torch.__version__)
print('Python version: ', python_version())
print('Sagemaker version: ', sagemaker.__version__)
print('Transformers version: ', transformers.__version__)
print('Datasets version: ', datasets.__version__)

# Pytorch version:  1.13.1
# Python version:  3.9.15
# Sagemaker version:  2.146.0
# Transformers version:  4.29.2
# Datasets version:  2.12.0

In [ ]:
# Sagemaker specific imports
import boto3
from sagemaker.huggingface import HuggingFace, TrainingCompilerConfig

# Huggingface specific imports
from datasets import Dataset, load_dataset, concatenate_datasets
from datasets.filesystems import S3FileSystem
from transformers import AutoTokenizer

In [ ]:
# Generic initilizations
model_id = 'google/flan-t5-xxl' 
workspace_bucket_name = 'gupshup-ml' # This s3 bucket is for storing datasets used for training.
s3_prefix = 'deepspeed' # s3 prefix at which train and test dataets will be saved. Ex - s3://gupshup-ml/deepspeed/train
model_name = model_id.split('/')[1]
save_model_s3_path = f's3://{workspace_bucket_name}/{s3_prefix}/{model_name}-deepspeed/' # s3 path where model artifacts gets stored (Used when trying to save using s5cmd)
experiment_name = model_name + '-finetuning'
job_name = 'qa-deepspeed-' + model_name

# Hyperparameters
epochs = 3                           # number of training epochs
per_device_train_batch_size = 5      # batch size for training
per_device_eval_batch_size = 5       # batch size for evaluation
gradient_accumulation_steps = 64
learning_rate = float('1e-4')  

In [ ]:
# Sagemaker specific
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f'Sagemaker Role ARN: {role}')
print(f'Sagemaker Bucket: {sess.default_bucket()}')
print(f'Sagemaker Session Region: {sess.boto_region_name}')

# Data

In [ ]:
# Load csv as a pandas dataframe
train_path = '../others/data/full_data/train/sniper_faq_session_train.csv'
train = pd.read_csv(train_path)
train['id'] = train.index
train = train[['id', 'input_text', 'output_text']]
print('Train:', train.shape)

test_path = '../others/data/full_data/test/sniper_faq_session_test.csv'
test = pd.read_csv(test_path)
test['id'] = test.index
test = test[['id', 'input_text', 'output_text']]
test.dropna(inplace=True)
print('Test:', test.shape)

# Create Dataset from pandas dataframes
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

dataset = datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset

# train = 6673, test = 1668 samples
# train.info(memory_usage='deep') # memory usage: 26.6 MB
# test.info(memory_usage='deep') # memory usage: 6.6 MB

In [ ]:
# Initialize tokenizer for a chosen model
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f'Model input names: {tokenizer.model_input_names}')
print(f'Model max length: {tokenizer.model_max_length}')

In [ ]:
%%time
# Data Processing

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset['train'], dataset['test']]).map(lambda x: tokenizer(x['input_text'], truncation=True), batched=True, remove_columns=['input_text', 'output_text'])
max_source_length = max([len(x) for x in tokenized_inputs['input_ids']])
print(f'Max source length: {max_source_length}')

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.'
tokenized_targets = concatenate_datasets([dataset['train'], dataset['test']]).map(lambda x: tokenizer(x['output_text'], truncation=True), batched=True, remove_columns=['input_text', 'output_text'])
max_target_length = max([len(x) for x in tokenized_targets['input_ids']])
print(f'Max target length: {max_target_length}')

def preprocess_function(sample, padding='max_length'):
    # add prefix to the input for t5
    inputs = sample['input_text']

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample['output_text'], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == 'max_length':
        labels['input_ids'] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels['input_ids']
        ]

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['id', 'input_text', 'output_text'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

In [ ]:
# Save the train and test datasets to s3
s3 = S3FileSystem()

training_input_path = f's3://{workspace_bucket_name}/{s3_prefix}/train'
print(f'Training input path: {training_input_path}')
tokenized_dataset['train'].save_to_disk(training_input_path, fs=s3)

test_input_path = f's3://{workspace_bucket_name}/{s3_prefix}/test'
print(f'Test input path: {test_input_path}')
tokenized_dataset['test'].save_to_disk(test_input_path, fs=s3)

# Fine-Tune

In [ ]:
# deepspeed related parameters for training job
deepspeed_parameters = {
  'deepspeed': './configs/ds_flan_t5_z3_offload', # path to deepspeed config file
  'training_script': './scripts/qa-deepspeed-s5cmd.py' # path to real training script, not entrypoint
}

hyperparameters, which are passed into the training job
training_hyperparameters={
  'model_id': model_id,                                        # pre-trained model
  'epochs': epochs,                                            # number of training epochs
  'per_device_train_batch_size': per_device_train_batch_size,  # batch size for training
  'per_device_eval_batch_size': per_device_eval_batch_size,    # batch size for evaluation
  'gradient_accumulation_steps': gradient_accumulation_steps,  # gradient accumulation steps
  'learning_rate': learning_rate,                              # learning rate used during training
  'generation_max_length': max_target_length,                  # max length of generated summary
  'save_model_s3_path': save_model_s3_path
}

print('Hyperparameters: \n', json.dumps(training_hyperparameters, indent=2, default=str))

In [ ]:
# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'launcher.py',     # deepspeed launcher script
    source_dir           = '.',               # directory which includes all the files needed for training
    instance_type        = 'ml.p4d.24xlarge', # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # IAM role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.17',            # the transformers version used in the training job
    pytorch_version      = '1.10',            # the pytorch_version version used in the training job
    py_version           = 'py38',            # the python version used in the training job
    hyperparameters      = {
      **training_hyperparameters,
      **deepspeed_parameters
    },                                        # the hyperparameter used for running the training job
    keep_alive_period_in_seconds = 1800,      # useful parameter when trying small changes right after failed job status. NOTE: This will keep the instance alive. Lookout for costs
)

data = {
    'train': training_input_path,
    'test' : test_input_path,
}
print(json.dumps(data, indent=2, default=str))

In [ ]:
%%time
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True) # Can also go with logs='None' parameter

In [ ]:
huggingface_estimator.hyperparameters()